# Rclone Installation on AppStream (Windows)

This report instructs how to build a Windows image with Rclone installed. Please follow the [Sleap-Windows-Report](./Sleap-Windows-Report.ipynb) for instructions on installing Sleap on Windows Image as well as a more detailed guide through the image building process for Windows.

- [Windows Build Image Instruction](#Windows-Build-Image-Instruction)
- [Windows User Instruction](#Windows-User-Instruction)
- [Optional: How to Transfer Files between S3 with Rclone](#Optional:-How-to-Transfer-files-between-S3-buckets-with-Rclone)

## Windows Build Image Instruction

See [Image Builder Guide](/AppStream_Image_Builder_Guide.ipynb) for an overview and explanation of each step, or see [Sleap-Windows-Report](./Sleap-Windows-Report.ipynb) for a more detailed run through each step. 

**Reminder**: Your image builder instance and/or fleet instances need to have an IAM role with S3 access attached, so that AppStream would create the `appstream-machine-role` profile for rclone to access the S3 buckets. 

### Admin User Install Package Step

    # Open Anaconda Prompt as Administrator: install choco
    @"%SystemRoot%\System32\WindowsPowerShell\v1.0\powershell.exe" -NoProfile -InputFormat None -ExecutionPolicy Bypass -Command "iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))" && SET "PATH=%PATH%;%ALLUSERSPROFILE%\chocolatey\bin"
    
    # Open Anaconda Prompt as usual:
    choco install rclone -y
    choco install winfsp -y
    
    # Create file C:\rclone\rclone.conf with contents:
    [s3]
    type = s3
    provider = AWS
    env_auth = true
    region = us-west-2
    profile = appstream_machine_role

Create Session Scripts to automate mounting buckets when users start an instance. Optionally, you can designate a location for log files for the mount command. 

    # Create empty directory for log files
    mkdir c:\rclone\logs
    
    # Create C:\rclone\mount.bat file
    @echo off
    set RCLONE_PATH="C:\ProgramData\chocolatey\bin\rclone.exe"
    set MOUNT_SOURCE=s3:aind-appstream-data-dev-temporary
    set MOUNT_DESTINATION="C:\s3-mount"
    set LOG_FILE="C:\rclone\logs\sync_files.txt"
    set RCLONE_CONFIG="C:\rclone\rclone.conf"

    %RCLONE_PATH% mount %MOUNT_SOURCE% %MOUNT_DESTINATION% --no-console --log-file %LOG_FILE% --config %RCLONE_CONFIG%
    @echo on
    
    # Open Anaconda Prompt as administrator, Edit contents of /appstream/SessionScripts/config.json 
        {
      "SessionStart": {
        "executables": [
          {
            "context": "user",
            "filename": "C:\\rclone\\mount.bat,
            "arguments": "",
            "s3LogEnabled": false
          }
        ],
        "waitingTime": 30
      },

### Template User Default Settings Step
    
`setx RCLONE_CONFIG 'C:\rclone\rclone.conf'`

The default rclone config file location is set as an environment variable, so that users can mount buckets without setting the config file flag. However, this will not be necessary in deployment since users are not expected to mount buckets on their own

### Test User Testing Step

You may mount and unmount buckets to test rclone, but do not expect the session script to be in effect at this phase. The session scripts will take effect when user instances are initiated. 

    rclone mount s3:aind-appstream-data-dev-temporary MOUNT_PATH --no-console
    taskkill /F /IM rclone.exe
    
    
### Admin User Optimization Step

Nothing extra is required. 


## Windows User Instruction


Here are some instructions if the user may wish to mount S3 buckets via rclone. In deployment, common buckets would already be installed in the image. 
 
    
### Mount and unmount s3 bucket (Optional)
    
The difference between using rclone in Linux and Windows is that in Windows, you should *not* create the mount directory beforehand, and the daemon flag is not available so background process is realize by the ampersand. The config file path must be set manually by the flag `--config-file c:\rclone\rclone.conf` or by setting the environment variable `RCLONE_CONFIG`, as done in this demo. The `--no-console` flag can be used to hide the console that runs the rclone command, the user would then have to use `taskkill /F /IM rclone.exe` to kill all rclone instances. 

    # mount s3 bucket in foreground
    rclone mount s3:aind-appstream-data-dev-temporary MOUNT_PATH
    
    # unmount s3 bucket in foreground
    [ctrl] + [C]
    
    # mount s3 bucket in background
    rclone mount s3:aind-appstream-data-dev-temporary MOUNT_PATH --no-console
    
    # unmount all s3 buckets in background
    taskkill /F /IM rclone.exe

### Optional: How to Transfer files between S3 buckets with Rclone

To test the performance of Sleap on AppStream instances, you may wish to analyze some large video files on S3. During this demo, I used Rclone to transfer a large video file to our appstream bucket for testing. 

**Transfer S3 data between buckets with 2 accounts**

In rclone.conf file, enter two AWS credentials with names `remoteprod` and `remotetest`. `remoteprod` credentials have read access to the video file, and `remotetest` credentials have write access to the appstream bucket. 

    git clone https://github.com/rclone/rclone.git
    cd rclone
    go build
    
    ./rclone copy "remoteprod:aind-behavior-data/pose_estimation_training/DLC annotation/1051155866_524760_20200917.behavior-Shailaja-2023-06-19/videos/1051155866_524760_20200917.behavior.mp4" "remotetest:aind-appstream-data-dev-temporary" --config rclone.conf

    ./rclone ls remotetest:aind-appstream-data-dev-temporary --config rclone.conf

    ./rclone ls "remoteprod:aind-behavior-data/pose_estimation_training/DLC annotation/1051155866_524760_20200917.behavior-Shailaja-2023-06-19/videos" --config rclone.conf